<a href="https://colab.research.google.com/github/hoangcuongnguyen2001/RAG_lessons/blob/main/Final_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bài thi cuối khóa**

Đây là bài kiểm tra khả năng của các bạn về việc xây dựng và sử dụng hệ thống RAG. Các bạn có thể dùng bất kỳ LLM nào cho bài này, miễn sao các bạn có thể trả về kết quả từ LLM là được.

Note: Tài liệu các bạn sẽ dùng trong bài test này là một bài báo về kinh tế học: *Trade, Growth and the Size of Countries*, của Alberto Alesina, Enrico Spolaore, Romain Wacziarg (2005). Với những bạn muốn tìm hiểu thêm, các bạn có thể tham khảo trên đây: https://www.sciencedirect.com/science/article/abs/pii/S1574068405010233


In [1]:
!pip install --upgrade huggingface_hub==0.23.5
!pip install --upgrade peft==0.12
!pip install llama-index bitsandbytes accelerate llama-index-llms-huggingface llama-index-embeddings-huggingface
!pip install --upgrade sentence-transformers
!pip install llama-index-postprocessor-colbert-rerank

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.27.0
    Uninstalling huggingface-hub-0.27.0:
      Successfully uninstalled huggingface-hub-0.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.14.0 requires huggingface-hub>=0.25.0, but you have huggingface-hub 0.23.5 which is incompatible.
transformers 4.47.1 requires huggingface-hub<1.0,>=0.24.0, but you have huggingface-hub 0.23.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 26.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.23.5
    Uninstalling huggingface-hub-0.23.5:
      Successfully uninstalled hu

In [15]:
!pip install pandas scikit-learn

In [2]:
!pip install pypdf

In [3]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

reader = SimpleDirectoryReader(input_files=["/content/Size_of_Countries.pdf"])
data = reader.load_data()


In [5]:
## Đánh giá cho test


from llama_index.core import PromptTemplate
from llama_index.postprocessor.colbert_rerank import ColbertRerank

system_prompt = "Please answer the questions in a truthful and reliable way."

query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [6]:
## Đánh giá cho test

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base", trust_remote_code=True)

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [7]:
# Đánh giá cho test

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    token=hf_token,
)

stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [8]:
# Đánh giá cho test

import torch
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = HuggingFaceLLM(
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    context_window=8192,
    max_new_tokens=512,
    model_kwargs={
        "token": hf_token,
        "torch_dtype": torch.bfloat16,
        "quantization_config": quantization_config
    },
    generate_kwargs={
        "do_sample": False,
        "temperature": 0.05,
        "top_p": 0.3,

    },
    tokenizer_name="meta-llama/Meta-Llama-3-8B-Instruct",
    tokenizer_kwargs={"token": hf_token},
    stopping_ids=stopping_ids,
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [9]:


from llama_index.core import Settings


Settings.embed_model = embed_model

Settings.chunk_size = 1024

Settings.llm = llm

In [10]:
## Đánh giá cho test

index = VectorStoreIndex.from_documents(data)

colbert_reranker = ColbertRerank(
    top_n=3,
    model="colbert-ir/colbertv2.0",
    tokenizer="colbert-ir/colbertv2.0",
    keep_retrieval_score=True,
)

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [11]:
## Đánh giá cho test

query_engine = index.as_query_engine(similarity_top_k=3, streaming=True, node_postprocessors=[colbert_reranker],)



In [12]:
response = query_engine.query(r'''
According to this article, what are the benefits of size in terms of population for countries?
''')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.05` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [13]:
## Đánh giá cho test
print(response)

According to the article, the main benefits of size in terms of population for countries are:

1. Economies of scale in the production of public goods, where the per capita cost of many public goods is lower in larger countries.
2. A larger country is less subject to foreign aggression, and smaller countries may have to spend proportionally more for defense.
3. Larger countries can better internalize cross-regional externalities by centralizing the provision of public goods that involve strong externalities.
4. Larger countries can provide insurance to regions affected by imperfectly correlated shocks.
5. Larger countries can build redistributive schemes from richer to poorer regions, achieving distributions of after-tax income that would not be available to individual regions acting independently.
6. The role of market size, where larger countries can create a limit on specialization, and the extent of the market can create a limit on the accumulation of human capital and the transmis

Lưu ý, sau khi có câu trả lời ở trên các bạn cần copy và dán câu trả lời này xuống mục *user_result* ở cell dưới. Nếu không, hệ thống sẽ không thể tính toán độ chính xác của câu trả lời mà LLM trả về.

In [17]:
# Đánh giá cho test

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Hãy copy câu trả lời (response) ở trên và dán vào đây:
user_result = r'''

 '''
actual_result = r'''
The main beneﬁts from size in terms of population are the following:
(1) There are economies of scale in the production of public goods. The per capita
cost of many public goods is lower in larger countries, where more taxpayers pay
for them.

(2) A larger country (both in terms of population and national product) is less subject
to foreign aggression. Thus, safety is a public good that increases with country
size. Also, and related to the size of government argument above, smaller coun-
tries may have to spend proportionally more for defense than larger countries
given economies of scale in defense spending.

(3) Larger countries can better internalize cross-regional externalities by centralizing
the provision of those public goods that involve strong externalities.
(4) Larger countries are better able to provide insurance to regions affected by im-
perfectly correlated shocks.

(5) Larger countries can build redistributive schemes from richer to poorer regions,
therefore achieving distributions of after tax income which would not be avail-
able to individual regions acting independently.

(6) Finally, the role of market size is the issue on which we focus most in this arti-
cle. Adam Smith (1776) already had the intuition that the extent of the market
creates a limit on specialization. More recently, a well established literature from
Romer (1986), Lucas (1988) to Grossman and Helpman (1991) has emphasized
the beneﬁts of scale in light of positive externalities in the accumulation of hu-
man capital and the transmission of knowledge, or in light of increasing returns
to scale embedded in technology or knowledge creation.

'''

documents = [user_result, actual_result]
count_vectorizer = CountVectorizer(stop_words="english")
count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(documents)

doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(
   doc_term_matrix,
   columns=count_vectorizer.get_feature_names_out(),
   index=["user_result", "actual_result"],
)

print(cosine_similarity(df, df))

AttributeError: 'StreamingResponse' object has no attribute 'lower'